In [43]:
import glob
import re
from functools import reduce
from tqdm.notebook import tqdm
import geocoder
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

In [44]:
DATA_PATH = "all_data/data/"

In [45]:
data = []

for filename in glob.glob(f"{DATA_PATH}offers*.json"):
    with open(filename, 'r') as file:
        offers = pd.read_json(file, orient='index')
        data.append(offers)

df = pd.concat(data)
df = df.reset_index()
df = df.rename(columns={'index': 'url', 'benefit-work-modes': 'work_modes', 'benefit-remote-recruitment': 'remote',
                        'benefit-employment-type-name': 'employment_type', 'benefit-work-schedule': 'schedule',
                        'benefit-contracts': 'contract'})
df

,url,work_modes,remote,employment_type,schedule,contract,locations,location_url,duties,requirements,offered,about_us,salary,benefits
0,https://www.pracuj.pl/praca/inventory-manager-...,,remote recruitment,team manager,full-time,contract of employment,"[Kraków, Kraków, Lesser Poland]",,[Analyze data to provide visibility of missed ...,[University degree in a technical or analytica...,[Chance to become a part of dynamically growin...,zooplus Polska Sp. z o.o.\nNumber 1 in Europe!...,,
1,https://www.pracuj.pl/praca/financial-analyst-...,,remote recruitment,"specialist (mid / regular), senior specialist ...",full-time,contract of employment,"[Warszawa, Warszawa, Masovian]",,[Support in planning (business plan and rollin...,"[University degree with Controlling, Finance, ...",[Exciting work in an international environment...,Technicolor Polska Sp. z o.o.\nOur Strength is...,,
2,https://www.pracuj.pl/praca/gl-accountant-wars...,,remote recruitment,"specialist (mid / regular), senior specialist ...",full-time,contract of employment,"[Warszawa, Warszawa, Masovian]",,[Performing general accounting and reporting t...,[Professional experience in the area of accoun...,[Exciting work in an international environment...,Technicolor Polska Sp. z o.o.\nOur Strength is...,,
3,https://www.pracuj.pl/praca/monter-urzadzen-el...,,rekrutacja zdalna,specjalista (mid / regular),pełny etat,umowa o pracę,"[aleja Grunwaldzka 103a, Gdańsk, Gdańsk, pomor...",https://www.google.pl/maps/search/54.379585266...,[Wykonywanie prac elektrycznych zgodnie z otrz...,[Wykształcenie techniczne - średnie lub zawodo...,[],MACGREGOR POLAND SP Z O O\nMacGregor shapes th...,,
4,https://www.pracuj.pl/praca/operator-maszyn-i-...,praca stacjonarna,,pracownik fizyczny,pełny etat,umowa o pracę,"[Trzciniec (pow. bydgoski), Trzciniec (pow. by...",,[Bieżąca obsługa maszyn i urządzeń sterowanych...,"[Wykształcenie min. zawodowe techniczne,, Dośw...",[Pracę w stabilnej firmie o ugruntowanej pozyc...,Xella Polska Sp. z o.o.\nXella Polska jest lid...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39441,https://www.pracuj.pl/praca/kierowca-miedzynar...,,,pracownik fizyczny,pełny etat,umowa o pracę,"[Tychy, Tychy, śląskie]",,"[KIEROWCA C+E MIĘDZYNARODÓWKA, DOKUPILIŚMY NOW...",[minimum pół roku doświadczenia w pracy jako k...,[legalne zatrudnienie w oparciu o umowę o prac...,,8 000‐8 500 zł brutto / mies.,8 000‐8 500 zł brutto / mies.
39442,"https://www.pracuj.pl/praca/monter-reda,oferta...",praca stacjonarna,,pracownik fizyczny,pełny etat,umowa o pracę,"[Wejherowska 108, Reda, Reda, pomorskie]",https://www.google.pl/maps/search/54.603878021...,[diagnozowanie oraz naprawa urządzeń klimatyza...,"[doświadczenie na podobnym stanowisku (3 lat),...","[pracę w dynamicznie rozwijającej się firmie,,...",FIRMA ACTIVE CLIMA\nSpecjalizujemy się w nowoc...,"30,00‐40,00 zł brutto / godz.","30,00‐40,00 zł brutto / godz."
39443,https://www.pracuj.pl/praca/doradca-ds-nieruch...,praca hybrydowa,,"specjalista (mid / regular), starszy specjalis...",pełny etat,"umowa zlecenie, kontrakt b2b","[Marszałkowska 87, Warszawa, Warszawa, mazowie...",https://www.google.pl/maps/search/52.227344512...,"[Praca w otwartym, pozytywnym zespole., Elasty...","[Kogo szukamy?, osób energicznych i otwartych,...","[pracę w elastycznych godzinach, możliwych do ...",Firma działająca na rynku nieruchomości poszuk...,UMOWA ZLECENIE\n5 000‐20 000 zł brutto / mies....,UMOWA ZLECENIE\n5 000‐20 000 zł brutto / mies....
39444,https://www.pracuj.pl/praca/kurier-bronisze-po...,,,pracownik fizyczny,pełny etat,"umowa o pracę, umowa zlecenie","[Świerkowa 1A, Bronisze (pow. warszawski zacho...",https://www.google.pl/maps/search/52.217609405...,[Doręczanie i odbiór przesyłek na wyznaczonym ...,"[Prawo jazdy kat. B, Doświadczenie w prowadzen...",[Stabilne zatrudnienie nawet podczas kolejnego...,Zatrudnię Kuriera do współpracy w firmie INPOS...,5 500‐7 000 zł brutto / mies.,5 500‐7 000 zł brutto / mies.


In [46]:
GOOGLE_MAPS_COORDINATES_PATTERN = re.compile(r'(?P<y>\d*.\d*),(?P<x>\d*.\d*)')

def infer_location(df_row):
  try:
    location = re.sub("[\(\[].*?[\)\]]", "", ", ".join(df_row['locations']))
    g = geocoder.osm(location)
    osm_dict = g.osm

    if df_row['location_url'] != '':
      x, y = re.search(GOOGLE_MAPS_COORDINATES_PATTERN, df_row['location_url']).group('x', 'y')
      osm_dict.update({'x': x, 'y': y})

    with open ("pracuj_inferred_locations_only_partial.csv", "a+") as file:
      file.write(str(osm_dict))
      file.write('\n')

    return osm_dict
  except Exception:
    return None

In [5]:
tqdm.pandas()
df['inferred_location'] = df.progress_apply(infer_location, axis=1)
df

  0%|          | 0/39446 [00:00<?, ?it/s]

Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Warszawa%2C+Warszawa%2C+Masovian&format=jsonv2&addressdetails=1&limit=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6b412ff040>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
Status code Unknown from https://nominatim.openstreetmap.org/search: ERROR - HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=aleja+Marsza%C5%82ka+J%C3%B3zefa+Pi%C5%82sudskiego+86%2C+Widzew%2C+%C5%81%C3%B3d%C5%BA%2C+%C5%81%C3%B3d%C5%BA%2C+%C5%81%C3%B3d%C5%BA&format=jsonv2&addressdetails=1&limit=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6b412ff3d0>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution'))
Status code Unknown 

,url,work_modes,remote,employment_type,schedule,contract,locations,location_url,duties,requirements,offered,about_us,salary,benefits,infered_location
0,https://www.pracuj.pl/praca/inventory-manager-...,,remote recruitment,team manager,full-time,contract of employment,"[Kraków, Kraków, Lesser Poland]",,[Analyze data to provide visibility of missed ...,[University degree in a technical or analytica...,[Chance to become a part of dynamically growin...,zooplus Polska Sp. z o.o.\nNumber 1 in Europe!...,,,"{'x': 19.9420309, 'y': 50.0636007, 'addr:stree..."
1,https://www.pracuj.pl/praca/financial-analyst-...,,remote recruitment,"specialist (mid / regular), senior specialist ...",full-time,contract of employment,"[Warszawa, Warszawa, Masovian]",,[Support in planning (business plan and rollin...,"[University degree with Controlling, Finance, ...",[Exciting work in an international environment...,Technicolor Polska Sp. z o.o.\nOur Strength is...,,,"{'x': 21.071432235636493, 'y': 52.2337172, 'ad..."
2,https://www.pracuj.pl/praca/gl-accountant-wars...,,remote recruitment,"specialist (mid / regular), senior specialist ...",full-time,contract of employment,"[Warszawa, Warszawa, Masovian]",,[Performing general accounting and reporting t...,[Professional experience in the area of accoun...,[Exciting work in an international environment...,Technicolor Polska Sp. z o.o.\nOur Strength is...,,,"{'x': 21.071432235636493, 'y': 52.2337172, 'ad..."
3,https://www.pracuj.pl/praca/monter-urzadzen-el...,,rekrutacja zdalna,specjalista (mid / regular),pełny etat,umowa o pracę,"[aleja Grunwaldzka 103a, Gdańsk, Gdańsk, pomor...",https://www.google.pl/maps/search/54.379585266...,[Wykonywanie prac elektrycznych zgodnie z otrz...,[Wykształcenie techniczne - średnie lub zawodo...,[],MACGREGOR POLAND SP Z O O\nMacGregor shapes th...,,,"{'x': '18.605209350585938', 'y': '54.379585266..."
4,https://www.pracuj.pl/praca/operator-maszyn-i-...,praca stacjonarna,,pracownik fizyczny,pełny etat,umowa o pracę,"[Trzciniec (pow. bydgoski), Trzciniec (pow. by...",,[Bieżąca obsługa maszyn i urządzeń sterowanych...,"[Wykształcenie min. zawodowe techniczne,, Dośw...",[Pracę w stabilnej firmie o ugruntowanej pozyc...,Xella Polska Sp. z o.o.\nXella Polska jest lid...,,,"{'x': 17.9480419, 'y': 53.0858024, 'addr:stree..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39441,https://www.pracuj.pl/praca/kierowca-miedzynar...,,,pracownik fizyczny,pełny etat,umowa o pracę,"[Tychy, Tychy, śląskie]",,"[KIEROWCA C+E MIĘDZYNARODÓWKA, DOKUPILIŚMY NOW...",[minimum pół roku doświadczenia w pracy jako k...,[legalne zatrudnienie w oparciu o umowę o prac...,,8 000‐8 500 zł brutto / mies.,8 000‐8 500 zł brutto / mies.,"{'x': 18.9641781, 'y': 50.1357681, 'addr:stree..."
39442,"https://www.pracuj.pl/praca/monter-reda,oferta...",praca stacjonarna,,pracownik fizyczny,pełny etat,umowa o pracę,"[Wejherowska 108, Reda, Reda, pomorskie]",https://www.google.pl/maps/search/54.603878021...,[diagnozowanie oraz naprawa urządzeń klimatyza...,"[doświadczenie na podobnym stanowisku (3 lat),...","[pracę w dynamicznie rozwijającej się firmie,,...",FIRMA ACTIVE CLIMA\nSpecjalizujemy się w nowoc...,"30,00‐40,00 zł brutto / godz.","30,00‐40,00 zł brutto / godz.","{'x': '18.30394172668457', 'y': '54.6038780212..."
39443,https://www.pracuj.pl/praca/doradca-ds-nieruch...,praca hybrydowa,,"specjalista (mid / regular), starszy specjalis...",pełny etat,"umowa zlecenie, kontrakt b2b","[Marszałkowska 87, Warszawa, Warszawa, mazowie...",https://www.google.pl/maps/search/52.227344512...,"[Praca w otwartym, pozytywnym zespole., Elasty...","[Kogo szukamy?, osób energicznych i otwartych,...","[pracę w elastycznych godzinach, możliwych do ...",Firma działająca na rynku nieruchomości poszuk...,UMOWA ZLECENIE\n5 000‐20 000 zł brutto / mies....,UMOWA ZLECENIE\n5 000‐20 000 zł brutto / mies....,"{'x': '21.012590408325195', 'y': '52.227344512..."
39444,https://www.pracuj.pl/praca/kurier-bronisze-po...,,,pracownik fizyczn

In [20]:
df.to_csv("pracuj_inferred_location.csv")

In [79]:
df_inferred = pd.read_csv("pracuj_inferred_location.csv", index_col=0)
df_inferred

,url,work_modes,remote,employment_type,schedule,contract,locations,location_url,duties,requirements,offered,about_us,salary,benefits,inferred_location
0,https://www.pracuj.pl/praca/inventory-manager-...,NaN,remote recruitment,team manager,full-time,contract of employment,"['Kraków', 'Kraków, Lesser Poland']",NaN,['Analyze data to provide visibility of missed...,['University degree in a technical or analytic...,['Chance to become a part of dynamically growi...,zooplus Polska Sp. z o.o.\nNumber 1 in Europe!...,NaN,NaN,"{'x': 19.9420309, 'y': 50.0636007, 'addr:stree..."
1,https://www.pracuj.pl/praca/financial-analyst-...,NaN,remote recruitment,"specialist (mid / regular), senior specialist ...",full-time,contract of employment,"['Warszawa', 'Warszawa, Masovian']",NaN,['Support in planning (business plan and rolli...,"['University degree with Controlling, Finance,...",['Exciting work in an international environmen...,Technicolor Polska Sp. z o.o.\nOur Strength is...,NaN,NaN,"{'x': 21.071432235636493, 'y': 52.2337172, 'ad..."
2,https://www.pracuj.pl/praca/gl-accountant-wars...,NaN,remote recruitment,"specialist (mid / regular), senior specialist ...",full-time,contract of employment,"['Warszawa', 'Warszawa, Masovian']",NaN,['Performing general accounting and reporting ...,['Professional experience in the area of accou...,['Exciting work in an international environmen...,Technicolor Polska Sp. z o.o.\nOur Strength is...,NaN,NaN,"{'x': 21.071432235636493, 'y': 52.2337172, 'ad..."
3,https://www.pracuj.pl/praca/monter-urzadzen-el...,NaN,rekrutacja zdalna,specjalista (mid / regular),pełny etat,umowa o pracę,"['aleja Grunwaldzka 103a, Gdańsk', 'Gdańsk, po...",https://www.google.pl/maps/search/54.379585266...,['Wykonywanie prac elektrycznych zgodnie z otr...,['Wykształcenie techniczne - średnie lub zawod...,[],MACGREGOR POLAND SP Z O O\nMacGregor shapes th...,NaN,NaN,"{'x': '18.605209350585938', 'y': '54.379585266..."
4,https://www.pracuj.pl/praca/operator-maszyn-i-...,praca stacjonarna,NaN,pracownik fizyczny,pełny etat,umowa o pracę,"['Trzciniec (pow. bydgoski)', 'Trzciniec (pow....",NaN,['Bieżąca obsługa maszyn i urządzeń sterowanyc...,"['Wykształcenie min. zawodowe techniczne,', 'D...",['Pracę w stabilnej firmie o ugruntowanej pozy...,Xella Polska Sp. z o.o.\nXella Polska jest lid...,NaN,NaN,"{'x': 17.9480419, 'y': 53.0858024, 'addr:stree..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39441,https://www.pracuj.pl/praca/kierowca-miedzynar...,NaN,NaN,pracownik fizyczny,pełny etat,umowa o pracę,"['Tychy', 'Tychy, śląskie']",NaN,"['KIEROWCA C+E MIĘDZYNARODÓWKA', 'DOKUPILIŚMY ...",['minimum pół roku doświadczenia w pracy jako ...,['legalne zatrudnienie w oparciu o umowę o pra...,NaN,8 000‐8 500 zł brutto / mies.,8 000‐8 500 zł brutto / mies.,"{'x': 18.9641781, 'y': 50.1357681, 'addr:stree..."
39442,"https://www.pracuj.pl/praca/monter-reda,oferta...",praca stacjonarna,NaN,pracownik fizyczny,pełny etat,umowa o pracę,"['Wejherowska 108, Reda', 'Reda, pomorskie']",https://www.google.pl/maps/search/54.603878021...,['diagnozowanie oraz naprawa urządzeń klimatyz...,['doświadczenie na podobnym stanowisku (3 lat)...,"['pracę w dynamicznie rozwijającej się firmie,...",FIRMA ACTIVE CLIMA\nSpecjalizujemy się w nowoc...,"30,00‐40,00 zł brutto / godz.","30,00‐40,00 zł brutto / godz.","{'x': '18.30394172668457', 'y': '54.6038780212..."
39443,https://www.pracuj.pl/praca/doradca-ds-nieruch...,praca hybrydowa,NaN,"specjalista (mid / regular), starszy specjalis...",pełny etat,"umowa zlecenie, kontrakt b2b","['Marszałkowska 87, Warszawa', 'Warszawa, mazo...",https://www.google.pl/maps/search/52.227344512...,"['Praca w otwartym, pozytywnym zespole.', 'Ela...","['Kogo szukamy?', 'osób energicznych i otwarty...","['pracę w elastycznych godzinach, możliwych do...",Firma działająca na rynku nieruchomości poszuk...,UMOWA ZLECENIE\n5 000‐20 000 zł brutto / mies....,UMOWA ZLECENIE\n5 000‐20 000 zł brutto / mies....,"{'x': '21.012590408325195', 'y':

In [84]:
from time import sleep


def infer_location_again(df_row):
  if pd.isnull(df_row['inferred_location']):
      sleep(3)
      df_row = df_row.copy()
      df_row['locations'] = df_row['locations'][0]
      return infer_location(df_row)
  else:
    return df_row['inferred_location']

In [85]:
tqdm.pandas()
df_inferred['inferred_location'] = df_inferred.progress_apply(infer_location_again, axis=1)

  0%|          | 0/39446 [00:00<?, ?it/s]

In [98]:
df_inferred.to_csv("pracuj_inferred_location_again.csv")

In [99]:
df_inferred_again = pd.read_csv("pracuj_inferred_location_again.csv", index_col=0)
df_inferred_again[pd.isnull(df_inferred_again['inferred_location'])]

,url,work_modes,remote,employment_type,schedule,contract,locations,location_url,duties,requirements,offered,about_us,salary,benefits,inferred_location
27,https://www.pracuj.pl/praca/elektryk-technik-k...,praca stacjonarna,rekrutacja zdalna,pracownik fizyczny,pełny etat,umowa o pracę,"['Stuttgart , Niemcy', 'zagranica']",NaN,['Kontrola urządzeń przenośnych i stacjonarnyc...,['Wykształcenie kierunkowe w zawodzie: Technik...,['Atrakcyjne wynagrodzenie oraz premię za efek...,OMS Prüfservice GmbH\nOMS Prüfservice GmbH jes...,9 000‐13 000 zł brutto / mies.,9 000‐13 000 zł brutto / mies.,NaN
31,https://www.pracuj.pl/praca/montazysta-nawierz...,NaN,NaN,pracownik fizyczny,pełny etat,umowa o pracę,"['Bergen, Norwegia', 'zagranica']",NaN,['Układanie w pełni formowanego podkładu z gum...,['Doświadczenie w układaniu nawierzchni gumowy...,['W Jobzone pracujemy zgodnie z wizja bycia na...,Jobzone Bergen AS\nPolacy dobrze się czują w N...,NaN,NaN,NaN
32,https://www.pracuj.pl/praca/monter-rusztowan-f...,NaN,NaN,pracownik fizyczny,pełny etat,umowa o pracę,"['Norwegia', 'zagranica']",NaN,"['Montaż i demontaż rusztowań.', 'Dbanie o to,...",['Certyfikat kompetencji montera rusztowań (§ ...,"['Możliwość pracy w systemie rotacyjnym.', 'Za...",Tak się u nas pracuje,NaN,NaN,NaN
42,https://www.pracuj.pl/praca/asystent-w-dziale-...,NaN,NaN,asystent,pełny etat,umowa o pracę,"['Ostrów Wielkopolski', 'Kalisz, wielkopolskie']",https://www.google.pl/maps/search/51.767279900...,[],[],[],NaN,NaN,NaN,NaN
55,https://www.pracuj.pl/praca/automatyk-programi...,NaN,NaN,specjalista (mid / regular),pełny etat,umowa o pracę,"['Północna 15/19, Fabryczna, Wrocław', 'Wrocła...",https://www.google.pl/maps/search/51.162128448...,"['tworzenie aplikacji SCADA/HMI', 'uruchamiani...",['doświadczenie w pracy z systemami typu SCADA...,"['udział w ciekawych projektach, dających możl...",PROCOM SYSTEM S.A.\nJesteśmy firmą od lat licz...,7 000‐9 000 zł brutto / mies.,7 000‐9 000 zł brutto / mies.,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39313,"https://www.pracuj.pl/praca/dekarz-belgia,ofer...",praca stacjonarna,rekrutacja zdalna,pracownik fizyczny,pełny etat,umowa o pracę,"['Belgia', 'zagranica']",NaN,"['renowacje dachów skośnych i płaskich', 'prac...","['doświadczenie w zakresie dekarstwa', 'samodz...","['legalne zatrudnienie na terenie Belgii', 'um...",Firma polska działająca ponad 10 lat na rynku ...,12 000‐13 000 zł brutto / mies.,12 000‐13 000 zł brutto / mies.,NaN
39319,"https://www.pracuj.pl/praca/komisjoner-niemcy,...",praca stacjonarna,rekrutacja zdalna,pracownik fizyczny,pełny etat,umowa o pracę,"['Niemcy', 'zagranica']",NaN,"['załadunek i rozładunek, komisjonowanie', 'ro...",['gotowość do podjęcia pracy od zaraz'],"['pełny etat, praca w systemie 3 zmianowym', '...",EastRecruiting to niemiecka firma zatrudniając...,6 000‐7 000 zł brutto / mies.,6 000‐7 000 zł brutto / mies.,NaN
39326,"https://www.pracuj.pl/praca/elektryk-niemcy,of...",praca stacjonarna,rekrutacja zdalna,pracownik fizyczny,pełny etat,umowa o pracę,"['Niemcy', 'zagranica']",NaN,['Samodzielne wykonywanie prac konserwacyjnych...,['Ukończone szkolenie zawodowe / szkoła zawodo...,['Stabilne zatrudnienie w oparciu o niemiecką ...,EastRecruiting to miedzynarodowa Agencja Pracy...,"65,00‐70,00 zł brutto / godz.","65,00‐70,00 zł brutto / godz.",NaN
39371,https://www.pracuj.pl/praca/monter-michalowice...,NaN,NaN,pracownik fizyczny,pełny etat,"umowa o pracę, umowa o dzieło, umowa zlecenie,...","['Jesionowa 32B, Michałowice-Osiedle (pow. pru...",https://www.google.pl/maps/search/52.176410675...,['Aktualnie posiadamy wolne wakaty na stanowis...,"['Od kandydatów wymagamy:', 'zachowania kultur...",['stabilne zatrudnienie w firmie działającej n...,AIR-KLIMA to firma z niemal 30 letnią tradycją...,"UMOWA O PRACĘ, UMOWA O DZIEŁO, UMOWA ZLECENIE\...","UMOWA O PRACĘ, UMOWA O DZIEŁO, UMOWA ZLECENIE\...",NaN
